# Проектирование редуктора

# Установка библиотек

In [ ]:
%pip install matplotlib

# Импорт библиотек

In [2]:
from math import pi, prod
from matplotlib import pyplot as plt

In [3]:
def choice(value, values, method):
    """Выбор значения из списка по методу"""
    if method == "ceil":
        for v in values:
            if v >= value:
                return v
    elif method == "floor":
        for v in values[::-1]:
            if v <= value:
                return v
    else:
        raise ValueError(f"{method=} not in ('ceil', 'floor')")

# 1. Расчет привода

Привод = устройство, предназначенное для приведения в действие машины. В основном, привод - это двигатель, соединенный с передачей.

Расчет привода включает в себя выбор двигателя и определение его (привода)  кинематических и силовых параметров.

На Рисунке 1.1 изображен привод к ленточному конвейеру, состоящий из двигателя, соединительной муфты, редуктора, предохранительной муфты, соединенной с приводным валом.

![](./assets/images/1.1.png)

Исходными данными являются технические данные исполнительного механизма (барабана с лентой).

Натяжение ветвей конвейера | F1 , кН и F2 , кН
---|---
Скорость ленты | V, м/с
Диаметр барабана | D, м
Ресурс работы | Lh тыс.час.



In [4]:
F1 = 6.6 * 1000  # Н
F2 = 3.1 * 1000  # Н
v = 1.45  # м/с
D = 0.145  # м
Lh = 12 * 1000 * 360  # c

## 1.1 Выбор электродвигателя 

При выборе электродвигателя  необходимо определить требуемые мощность и частоту вращения вала двигателя, которые обеспечат работу привода с заданными характеристиками. 

Выбор требуемой мощности  электродвигателя  Рэд.т (кВт).
Мощность на приводном валу Рпр.в(4) (вал №4 на схеме привода).

In [5]:
P_пр_в = (F1 - F2) * v
print(f"{P_пр_в:.1f} Вт")

5075.0 Вт


Коэффициенты полезного действия и передаточные числа передач

Передача | КПД | u
---|---|---
Коническая закрытая | 0,95…0,97 | 1,0…6,3
Цепная открытая | 0.92…0,95 | 1,5…3,0
Ременная открытая | 0,94…0,96 | 2,0…4,0
Муфта соединительная | 0,98…0,99 | -
Муфта предохранительная | 0,99…0,995 | -

Примечание. Если в приводе имеются другие передачи необходимо учесть потерю мощности в общем КПД привода. 

In [126]:
eff_м = 0.98  # КПД муфты соединительной
eff_к = 0.97  # КПД конической передачи
eff_п = 0.99  # КПД  одной пары подшипников

In [248]:
eff_i = (eff_м, eff_к, eff_п)
print(f"{eff_i=}")

eff_i=(0.98, 0.97, 0.99)


Общий КПД привода

In [249]:
eff = prod(eff_i)
print(f"{eff:.4f}")

0.9411


Требуемая мощность электродвигателя  Рэд.тр.

In [271]:
P_эд_req = P_пр_в / eff
print(f"{P_эд_req=:.1f} Вт")

P_эд_req=5392.7 Вт


По значению Рэд.р. выбирается мощность двигателя Рэд.

Технические характеристики асинхронных трехфазных короткозамкнутых закрытых обдуваемых двигателей с нормальным пусковым моментом

![](./assets/images/engins.png)


In [251]:
P_ENGINE = (370, 550, 750, 1_100, 1_500, 2_200, 3_000, 4_000, 5_500, 7_500, 11_000, 15_000, 18_500, 22_000, 30_000)

In [272]:
P_эд = choice(P_эд_req, P_ENGINE, method="ceil")
print(f"{P_эд=} Вт")

P_эд=5500 Вт


In [253]:
n_эд = [750 * 2 * pi / 60, 1000 * 2 * pi / 60, 1500 * 2 * pi / 60, 3000 * 2 * pi / 60]
print(f"{n_эд} рад/c")

[78.53981633974483, 104.71975511965977, 157.07963267948966, 314.1592653589793] рад/c


In [254]:
n_эд = 965 * 2 * pi / 60

In [273]:
T_эд = P_эд_req / n_эд / 2 / pi
print(f"{T_эд=:.2f} Н*м")

T_эд=8.49 Н*м


In [274]:
d_эд = 28 / 1000

## 1.2 Выбор частоты вращения вала электродвигателя n_e

Частота вращения приводного вала

In [275]:
n_пр = v / (D / 2)
print(f"{n_пр} c^-1")

20.0 c^-1


Общее передаточное число привода

In [276]:
u_о = n_эд / n_пр
print(f"{u_о} c^-1")

5.052728184523584 c^-1


Выбираем по Табл.1.3  частоту вращения вала двигателя  и передаточные числа передач привода (Табл.1.2  и Табл.1.4) и марку двигателя (Табл.1.3).

1 ряд | 1,00 | 1,25 | 1,6 | 2,00 | 2,50 | 3,15 | 4,0 | 5,0 | 6,3 | 12,5 | 16 | 20 | 25 | 31,5 | 40 | 50 | 63 | 80
---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---
2 ряд | 1,12 | 1,40 | 1,8 | 2,24 | 2,8 | 3,55 | 4,5 | - | 5,6 | 14 | 18 | 22,4 | 26 | 35,5 | 45 | 56 | 71 | -

In [310]:
U1 = [1, 1.25, 1.6, 2, 2.5, 3.15, 4, 5, 6.3, 12.5, 16, 20, 25, 31.5, 40, 50, 63, 80]
U2 = [1.12, 1.4, 1.8, 2.24, 2.8, 3.55, 4.5, 5.6, 14, 18, 22.4, 26, 35.5, 45, 56, 71]
U = sorted(U1 + U2)

In [311]:
u_о = choice(u_о, U, "floor")
print(f"{u_о=}")

u_о=5


## 1.3 Кинематический и силовой расчет привода

In [313]:
n_shafts = 4

u_i = [1, 5, 1]

P_i = [P_эд_req]  # требуемая мощность
n_i = [n_эд]
T_i = [P_i[0] / n_i[0]]
d_i = [d_эд]

for i in range(n_shafts):
    if i != n_shafts - 1:
        P_i.append(P_i[i] * eff_i[i])
        n_i.append(n_i[i] / u_i[i])
    else:
        P_i.append(P_i[i] * 1)
        n_i.append(n_i[i] / 1)
    T_i.append(P_i[i + 1] / n_i[i + 1])

    if i == 0:
        c = 8
    elif i == 1:
        c = 6
    else:
        c = 5
    d_i.append(c / 1000 * (T_i[i]) ** (1 / 3))

    print(f"{i + 1}")
    print(f"{P_i[i]=:<10.1f} Вт")
    print(f"{n_i[i]=:<10.4f} рад/c")
    print(f"{T_i[i]=:<10.3f} Н*м")
    print(f"{d_i[i]=:<10.3f} м")

    print()

    if i != n_shafts - 1:
        print(f"{eff_i[i]=:<10}")
        print(f"{u_i[i]=:<10}")

        print()

1
P_i[i]=5392.7     Вт
n_i[i]=101.0546   рад/c
T_i[i]=53.364     Н*м
d_i[i]=0.028      м

eff_i[i]=0.98      
u_i[i]=1         

2
P_i[i]=5284.8     Вт
n_i[i]=101.0546   рад/c
T_i[i]=52.297     Н*м
d_i[i]=0.030      м

eff_i[i]=0.97      
u_i[i]=5         

3
P_i[i]=5126.3     Вт
n_i[i]=20.2109    рад/c
T_i[i]=253.638    Н*м
d_i[i]=0.022      м

eff_i[i]=0.99      
u_i[i]=1         

4
P_i[i]=5075.0     Вт
n_i[i]=20.2109    рад/c
T_i[i]=251.102    Н*м
d_i[i]=0.032      м

